# Import

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
seed = 42

import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from sklearn.model_selection import train_test_split

import random
random.seed(seed)
from sklearn.utils import shuffle

import matplotlib.pyplot as plt
import cv2
import numpy as np
import os
import glob
from PIL import Image
from imgaug import augmenters as iaa
from skimage.transform import resize
from skimage.metrics import structural_similarity as ssim
from tensorflow.keras.initializers import he_normal, glorot_uniform, glorot_normal
from tensorflow.keras.optimizers import Adam, SGD, AdamW
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.callbacks import TensorBoard, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.layers import GlobalAveragePooling2D, BatchNormalization, Input, Conv2D, MaxPooling2D, UpSampling2D, Dense, Reshape, Flatten, Dropout, Conv2DTranspose, LeakyReLU, LayerNormalization, MultiHeadAttention, Add, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras.losses import MeanSquaredError, BinaryCrossentropy, mean_squared_error
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn import svm
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

# Impostazioni GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
devices = tf.config.experimental.list_physical_devices()
print("Dispositivi visibili a TensorFlow:")
for device in devices:
    print(device)

import seaborn as sns
import matplotlib.patches as patches


## Import Ste

In [ ]:

Y_train=np.load("/kaggle/input/labels-finali/labels_array_03.npy")

X_train=np.load("/kaggle/input/immagini-corrette-bw/images_array_col_03_giuste_bw.npy")

## Import Ale

In [ ]:
X_train=np.load("/kaggle/input/datiandlabels/images_array_bw.npy")
Y_train=np.load("/kaggle/input/datiandlabels/labels_array_03.npy")
print(X_train.shape)
print(Y_train.shape)

print(sum(Y_train==0)) 
print(sum(Y_train==1))


# AUTOENCODER double loss

### augmentation con labels

In [ ]:


def augment_images_with_labels_01(images_array, labels_array, num_augmented_copies_class_0, num_augmented_copies_class_1):
    if images_array.ndim == 3:
        images_array = np.expand_dims(images_array, axis=-1)
    
    # Crea un oggetto ImageDataGenerator con le trasformazioni desiderate
    datagen = ImageDataGenerator(
        horizontal_flip=True,  # Ribaltamento orizzontale
        vertical_flip=True,    # Ribaltamento verticale
        rotation_range=10,     # Rotazione casuale tra -10 e 10 gradi
        width_shift_range=0.1, # Traslazione casuale del 10% su asse x
        height_shift_range=0.1,# Traslazione casuale del 10% su asse y
        zoom_range=0.2,        # Zoom casuale tra l'80% e il 120%
        brightness_range=[0.5, 0.8]  # Modifica casuale della luminosità tra l'80% e il 120%
    )

    # Liste per memorizzare le immagini e le etichette aumentate
    augmented_images = []
    augmented_labels = []

    # Indici per separare le immagini per classe
    indices_class_0 = np.where(labels_array == 0)[0]
    indices_class_1 = np.where(labels_array == 1)[0]

    # Seleziona le immagini per ciascuna classe
    images_class_0 = images_array[indices_class_0]
    labels_class_0 = labels_array[indices_class_0]
    images_class_1 = images_array[indices_class_1]
    labels_class_1 = labels_array[indices_class_1]

    # Funzione per aumentare le immagini di una classe specifica
    def augment_class_images(images, labels, num_augmented_copies):
        augmented_images = []
        augmented_labels = []
        for image, label in zip(images, labels):
            # Aggiungi l'immagine e l'etichetta originale
            augmented_images.append(image)
            augmented_labels.append(label)

            # Applica l'aumentazione per il numero specificato di copie
            image = np.expand_dims(image, 0)  # Espandi la dimensione per farla diventare (1, h, w, c)
            i = 0
            for batch in datagen.flow(image, batch_size=1):
                augmented_images.append(batch[0])
                augmented_labels.append(label)  # Aggiungi la stessa etichetta per l'immagine aumentata
                i += 1
                if i >= num_augmented_copies:
                    break
        return np.array(augmented_images), np.array(augmented_labels)

    # Aumenta le immagini della classe 0 e 1
    augmented_images_class_0, augmented_labels_class_0 = augment_class_images(images_class_0, labels_class_0, num_augmented_copies_class_0)
    augmented_images_class_1, augmented_labels_class_1 = augment_class_images(images_class_1, labels_class_1, num_augmented_copies_class_1)

    # Combina le immagini e le etichette aumentate delle due classi
    augmented_images = np.concatenate([augmented_images_class_0, augmented_images_class_1])
    augmented_labels = np.concatenate([augmented_labels_class_0, augmented_labels_class_1])

    # Stampa le dimensioni degli array
    print("Dimensione del vettore delle immagini aumentate:", augmented_images.shape)
    print("Dimensione del vettore delle etichette aumentate:", augmented_labels.shape)

    return augmented_images, augmented_labels




### split dati

In [ ]:
# Dati di esempio
# Assumendo che X_train e Y_train siano i tuoi dati, con X_train di shape (3000, 64, 64)

print(X_train.shape)
print(Y_train.shape)

print(sum(Y_train==0)) 
print(sum(Y_train==1))


X = X_train.reshape(-1, 64, 64, 1)
Y = Y_train

# Split dei dati in training, validation e test set
X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, test_size=0.3, random_state=42, shuffle=True, stratify=Y_train)
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=42, shuffle=True, stratify=Y_temp)

print(sum(Y_train==0)) 
print(sum(Y_train==1))

print(sum(Y_val==0)) 
print(sum(Y_val==1))

print(sum(Y_test==0)) 
print(sum(Y_test==1))

## preprocessing dati 
Aumentiamo i dati di train, abbiamo provato ad aumentare maggiormente la sottoclasse (1) 
Abbiamo tentato anche di aumentare leggermente il validation 

In [ ]:
X_train, Y_train = augment_images_with_labels_01(X_train, Y_train, 4, 8)
X_val, Y_val = augment_images_with_labels_01(X_val, Y_val, 2, 3)


X_train = shuffle(X_train, random_state = 58)
Y_train = shuffle(Y_train, random_state = 58)
X_val = shuffle(X_val, random_state = 99)
Y_val = shuffle(Y_val, random_state = 99)

print(X_train.shape)
print(Y_train.shape)
print(sum(Y_train==0)) 
print(sum(Y_train==1))

print(X_val.shape)
print(Y_val.shape)
print(sum(Y_val==0)) 
print(sum(Y_val==1))

X_train = X_train / 255.0
X_val = X_val / 255.0
X_test = X_test / 255.0


## Primo modello 3 strati convoluzionali
pesi salvati qui:  /Users/alessiamenozzi/Desktop/encoder_doubleloss_easy128.weights.h5

In [ ]:


lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-4,
    decay_steps=35000,
    decay_rate=0.9
)

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=100,         
    restore_best_weights=True, 
    min_delta=0.00001,
    verbose = True
)

## dizionari creati per gestire la doppia loss in modo che le loss vengano ottimizzate insieme
stored_values = {
    'y_true_class': None,
    'y_pred_class': None,
    'y_true_recon': None,
    'y_pred_recon': None
}


# Funzione per la loss combinata
def create_combined_loss(lambdadec, lambdatot):
    def combined_loss_diz(y_true, y_pred):
        if len(y_true.shape) == 2:  # Output del classificatore
            stored_values['y_true_class'] = y_true
            stored_values['y_pred_class'] = y_pred
        elif len(y_true.shape) == 4:  # Output del decoder (immagini)
            stored_values['y_true_recon'] = y_true
            stored_values['y_pred_recon'] = y_pred

        if stored_values['y_true_class'] is not None and stored_values['y_true_recon'] is not None:
            # perdita per il classificatore
            bce_loss = tf.keras.losses.BinaryCrossentropy()(stored_values['y_true_class'], stored_values['y_pred_class'])
            
            # perdita per il decoder
            mse_loss = tf.keras.losses.MeanSquaredError()(stored_values['y_true_recon'], stored_values['y_pred_recon'])
            bce_loss_re = tf.keras.losses.BinaryCrossentropy()(stored_values['y_true_recon'], stored_values['y_true_recon'])

            #ssim_loss = 1 - tf.reduce_mean(tf.image.ssim(stored_values['y_true_recon'], stored_values['y_pred_recon'], max_val=1.0))
            dec_loss= lambdadec * mse_loss + (1-lambdadec) * bce_loss_re
            
            ## loss combinata
            combined_loss_value = lambdatot * dec_loss + (1-lambdatot) * bce_loss
        
            stored_values['y_true_class'] = None
            stored_values['y_pred_class'] = None
            stored_values['y_true_recon'] = None
            stored_values['y_pred_recon'] = None
            
            return combined_loss_value

        return 0.0
    return combined_loss_diz



lossWeights={'classifier': 0.3, 'decoder': 0.7}


def create_encoder(input_shape):
    inputs = Input(shape=input_shape)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.001))(inputs)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Flatten()(x)
    encoded = Dense(128, activation='relu', kernel_regularizer=l2(0.001))(x)
    return Model(inputs, encoded, name="encoder")


def create_decoder(encoded_shape):
    inputs = Input(shape=encoded_shape)
    x = Dense(8 * 8 * 32, activation='relu', kernel_regularizer=l2(0.001))(inputs)
    x = Reshape((8, 8, 32))(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same', kernel_regularizer=l2(0.001))(x)
    return Model(inputs, decoded, name="decoder")


def create_classifier(encoded_shape):
    inputs = Input(shape=encoded_shape)
    classification = Dense(1, activation='sigmoid')(inputs)
    return Model(inputs, classification, name="classifier")

def build_autoencoder_classifier(input_shape, encoded_shape):
    encoder = create_encoder(input_shape)
    decoder = create_decoder(encoded_shape)
    classifier = create_classifier(encoded_shape)


    inputs = Input(shape=input_shape)

    # Encoding
    encoded = encoder(inputs)

    # Decoding
    reconstructed = decoder(encoded)

    # Classification
    classification = classifier(encoded)

    # Modello finale
    autoencoder_classifier = Model(inputs, [classification, reconstructed], name='autoencoder_classifier')

    autoencoder_classifier.compile(optimizer=Adam(learning_rate=1e-4),
                                   #loss_weights=lossWeights,
                                   #loss={'decoder': 'mse', 'classifier': 'bce'},
                                   loss=create_combined_loss(0.5, 0.4),
                                   metrics={'classifier': 'accuracy'})

    return autoencoder_classifier


input_shape = (64, 64, 1)
encoded_shape = (128,)


autoencoder_classifier = build_autoencoder_classifier(input_shape, encoded_shape)


autoencoder_classifier.summary()

# Addestramento del modello
history = autoencoder_classifier.fit(
    x=X_train,
    y=[Y_train, X_train],
    epochs=320,
    batch_size=128,
    validation_data=(X_val, [Y_val, X_val]),
    #callbacks=[early_stopping]
)


## Secondo Modello 7 strati convoluzionali


In [ ]:

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=100,          
    restore_best_weights=True, 
    min_delta=0.0001,
    verbose = True
)

stored_values = {
    'y_true_class': None,
    'y_pred_class': None,
    'y_true_recon': None,
    'y_pred_recon': None
}

tf.config.run_functions_eagerly(False)


def create_combined_loss(lambdadec, lambdatot):
    def combined_loss_diz(y_true, y_pred):
        if len(y_true.shape) == 2:  # Output del classificatore
            stored_values['y_true_class'] = y_true
            stored_values['y_pred_class'] = y_pred
        elif len(y_true.shape) == 4:  # Output del decoder (immagini)
            stored_values['y_true_recon'] = y_true
            stored_values['y_pred_recon'] = y_pred

        if stored_values['y_true_class'] is not None and stored_values['y_true_recon'] is not None:
            # perdita per il classificatore
            bce_loss = tf.keras.losses.BinaryCrossentropy()(stored_values['y_true_class'], stored_values['y_pred_class'])
            
            # perdita per il decoder
            mse_loss = tf.keras.losses.MeanSquaredError()(stored_values['y_true_recon'], stored_values['y_pred_recon'])
            bce_loss_re = tf.keras.losses.BinaryCrossentropy()(stored_values['y_true_recon'], stored_values['y_true_recon'])
            #ssim_loss = 1 - tf.reduce_mean(tf.image.ssim(stored_values['y_true_recon'], stored_values['y_pred_recon'], max_val=1.0))
            
            dec_loss= lambdadec * mse_loss + (1-lambdadec) * bce_loss_re
            
            combined_loss_value = lambdatot * dec_loss + (1-lambdatot) * bce_loss
        
            stored_values['y_true_class'] = None
            stored_values['y_pred_class'] = None
            stored_values['y_true_recon'] = None
            stored_values['y_pred_recon'] = None
            
            return combined_loss_value

        return 0.0
    return combined_loss_diz


lossWeights={'classifier': 0.3, 'decoder': 0.7}


def create_encoder(input_shape):
    inputs = Input(shape=input_shape)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(inputs)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = GlobalAveragePooling2D()(x)
    x = Flatten()(x)
    
    encoded = Dense(64, activation='relu', kernel_regularizer=l1(1e-5))(x)
    encoder_model = Model(inputs, encoded, name='encoder')
    return encoder_model

# Decoder
def create_decoder(encoded_shape):
    inputs = Input(shape=encoded_shape)
    x = Dense(1 * 1 * 512, activation='relu', kernel_regularizer=l1(1e-5))(inputs)
    x = Reshape((1, 1, 512))(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
    decoder_model = Model(inputs, decoded, name='decoder')
    return decoder_model

# Funzione per creare il classificatore
def create_classifier(encoded_shape):
    inputs = Input(shape=encoded_shape)
    classification = Dense(1, activation='sigmoid')(inputs)
    return Model(inputs, classification, name="classifier")

# Funzione per costruire l'intero modello Autoencoder-Classifier
def build_autoencoder_classifier(input_shape, encoded_shape):
    encoder = create_encoder(input_shape)
    decoder = create_decoder(encoded_shape)
    classifier = create_classifier(encoded_shape)

    # Input
    inputs = Input(shape=input_shape)

    # Encoding
    encoded = encoder(inputs)

    # Decoding
    reconstructed = decoder(encoded)

    # Classification
    classification = classifier(encoded)

    # Modello finale
    autoencoder_classifier = Model(inputs, [classification, reconstructed], name='autoencoder_classifier')

    # Compilazione del modello
    autoencoder_classifier.compile(optimizer=Adam(learning_rate=1e-4),
                                   #loss_weights=lossWeights,
                                   #loss={'decoder': 'mse', 'classifier': 'bce'},
                                   loss=create_combined_loss(0.5, 0.9),
                                   metrics={'classifier': 'accuracy'})

    return autoencoder_classifier

# Parametri del modello
input_shape = (64, 64, 1)
encoded_shape = (64,)

# Costruzione del modello
autoencoder_classifier = build_autoencoder_classifier(input_shape, encoded_shape)

# Visualizzazione del modello
autoencoder_classifier.summary()


In [ ]:
encoder = autoencoder_classifier.get_layer("encoder")
encoder.save_weights('encoder_doubleloss_easy64.weights.h5')

## Secondo Modello + trasformer

In [ ]:

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-4,
    decay_steps=10000,
    decay_rate=0.9
)

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',  
    patience=50,          
    restore_best_weights=True, 
    min_delta=0.0001,
    verbose = True
)

#lossWeights={'classifier': 0.3, 'decoder': 0.7}


stored_values = {
    'y_true_class': None,
    'y_pred_class': None,
    'y_true_recon': None,
    'y_pred_recon': None
}

#tf.config.run_functions_eagerly(False)


def create_combined_loss(lambdadec, lambdatot):
    def combined_loss_diz(y_true, y_pred):
        if len(y_true.shape) == 2:  
            stored_values['y_true_class'] = y_true
            stored_values['y_pred_class'] = y_pred
        elif len(y_true.shape) == 4:  
            stored_values['y_true_recon'] = y_true
            stored_values['y_pred_recon'] = y_pred

        if stored_values['y_true_class'] is not None and stored_values['y_true_recon'] is not None:
            bce_loss = tf.keras.losses.BinaryCrossentropy()(stored_values['y_true_class'], stored_values['y_pred_class'])
            
            mse_loss = tf.keras.losses.MeanSquaredError()(stored_values['y_true_recon'], stored_values['y_pred_recon'])
            bce_loss_re = tf.keras.losses.BinaryCrossentropy()(stored_values['y_true_recon'], stored_values['y_true_recon'])
            #ssim_loss = 1 - tf.reduce_mean(tf.image.ssim(stored_values['y_true_recon'], stored_values['y_pred_recon'], max_val=1.0))
            dec_loss= lambdadec * mse_loss + (1-lambdadec) * bce_loss_re
            

            combined_loss_value = lambdatot * dec_loss + (1-lambdatot) * bce_loss
        
            stored_values['y_true_class'] = None
            stored_values['y_pred_class'] = None
            stored_values['y_true_recon'] = None
            stored_values['y_pred_recon'] = None
            
            return combined_loss_value

        return 0.0
    return combined_loss_diz



# Blocco Transformer
def transformer_block(x, num_heads, ff_dim, rate=0.3):
    attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=x.shape[-1])(x, x)
    attn_output = Dropout(rate)(attn_output)
    out1 = Add()([x, attn_output])  
    out1 = LayerNormalization(epsilon=1e-6)(out1)

    ff_output = Dense(ff_dim, activation='relu')(out1)  
    ff_output = Dense(x.shape[-1])(ff_output)  
    ff_output = Dropout(rate)(ff_output)

    out2 = Add()([out1, ff_output])  
    return LayerNormalization(epsilon=1e-6)(out2)


def encoder(input_shape):
    inputs = Input(shape=input_shape)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(inputs)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = GlobalAveragePooling2D()(x)
    x = Flatten()(x)
    
    encoded = Dense(64, activation='relu', kernel_regularizer=l1(1e-5))(x)
    encoder_model = Model(inputs, encoded, name='encoder')
    return encoder_model


def decoder(encoded_shape):
    inputs = Input(shape=encoded_shape)
    x = Dense(1 * 1 * 512, activation='relu', kernel_regularizer=l1(1e-5))(inputs)
    x = Reshape((1, 1, 512))(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
    decoder_model = Model(inputs, decoded, name='decoder')
    return decoder_model


def classifier(encoded_shape):
    inputs = Input(shape=encoded_shape)
    classification = Dense(1, activation='sigmoid')(inputs)
    return Model(inputs, classification, name="classifier")


def build_autoencoder_classifier(input_shape, encoded_shape):
    encoder_model = encoder(input_shape)
    
    # Trasformo il bottleneck per simulare una sequenza (per il Transformer)
    x = Reshape((1, 64))(encoder_model.output)
    #x = transformer_block(x, num_heads=8, ff_dim=512)
    x = transformer_block(x, num_heads=6, ff_dim=256)
    #x = transformer_block(x, num_heads=8, ff_dim=512)
    x = Flatten()(x)
    

    decoder_model = decoder((64,))
    reconstructed = decoder_model(x)
    

    classifier_model = classifier((64,))
    classification = classifier_model(x)


    autoencoder_classifier = Model(encoder_model.input, [classification, reconstructed], name='autoencoder_classifier')
    

    autoencoder_classifier.compile(optimizer=Adam(learning_rate=lr_schedule), 
                                   loss={'decoder': 'bce'},
                                   #loss_weights=lossWeights,
                                   #loss=create_combined_loss(0.5, 0.4),
                                   metrics={'classifier': 'accuracy'})

    return autoencoder_classifier


input_shape = (64, 64, 1)
encoded_shape = (64,)


autoencoder_classifier = build_autoencoder_classifier(input_shape, encoded_shape)

autoencoder_classifier.summary()




In [ ]:

history = autoencoder_classifier.fit(
    x=X_train, 
    y=[Y_train, X_train],
    epochs=1000, 
    batch_size=256, 
    validation_data=(X_val, [Y_val, X_val]),
    callbacks=[early_stopping]
)

In [ ]:
encoder_input = autoencoder_classifier.input 
encoder_output = autoencoder_classifier.get_layer('flatten_3').output


encoder_model = Model(inputs=encoder_input, outputs=encoder_output, name='encoder')

encoder_model.save_weights('encoder_DoubleLoss_transformer.weights.h5')


## VALUTAZIONI ricostruzioni e classificazione

In [ ]:


# Valutazione sul test set CLASSIFICAZIONE
results = autoencoder_classifier.evaluate(X_test, Y_test)
print(f'Test Loss: {results[0]}, Test Accuracy: {results[1]}')


Y_test_pred, _  = autoencoder_classifier.predict(X_test)
#_, Y_test_pred  = autoencoder_classifier.predict(X_test)
# Soglia per la classificazione
Y_test_pred_binary = (Y_test_pred > 0.5).astype(int)


conf_matrix = confusion_matrix(Y_test, Y_test_pred_binary)
class_report = classification_report(Y_test, Y_test_pred_binary, target_names=['Class 0', 'Class 1'])


print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Class 0', 'Class 1'], yticklabels=['Class 0', 'Class 1'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()


val_loss = history.history['val_loss']
val_accuracy = history.history.get('val_classifier_accuracy', [])


print(f"Validation Loss: {val_loss[-1]}")
if val_accuracy:
    print(f"Validation Accuracy: {val_accuracy[-1]}")
else:
    print("Validation Accuracy not available in history.")


plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

if val_accuracy:
    plt.figure(figsize=(12, 6))
    plt.plot(history.history['classifier_accuracy'], label='Training Accuracy')
    plt.plot(val_accuracy, label='Validation Accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()


In [ ]:
import numpy as np
from skimage.metrics import structural_similarity as ssim

### VALUTAZIONE RICOSTRUZIONI

# Funzione per calcolare il SSIM medio
def calculate_mean_ssim(X_true, X_pred):
    ssim_values = []
    for i in range(len(X_true)):
        # Calcolo del SSIM per ciascuna immagine
        ssim_value = ssim(X_true[i].reshape(64, 64), X_pred[i].reshape(64, 64), data_range=1.0)
        ssim_values.append(ssim_value)
        
    std_ssim = np.std(ssim_values)
    mean_ssim= np.mean(ssim_values)
    return std_ssim, mean_ssim



_, X_test_reconstructed = autoencoder_classifier.predict(X_test)
#X_test_reconstructed, _ = autoencoder_classifier.predict(X_test)

# Calcolo del SSIM medio
std_ssim, mean_ssim = calculate_mean_ssim(X_test, X_test_reconstructed)
print(f'Mean SSIM on Test Set: {mean_ssim}')
print(f'Std SSIM on Test Set: {std_ssim}')

In [ ]:
## SSIM CON MASK
import numpy as np
from skimage.metrics import structural_similarity as ssim
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def find_bounding_box(image):
    # Trova i bordi del rettangolo contenente la placca
    rows = np.any(image, axis=1)
    cols = np.any(image, axis=0)
    y_min, y_max = np.where(rows)[0][[0, -1]]
    x_min, x_max = np.where(cols)[0][[0, -1]]
    return y_min, y_max, x_min, x_max


if X_test_reconstructed.shape[-1] == 1:
    X_test_reconstructed = np.squeeze(X_test_reconstructed, axis=-1)
if X_test.shape[-1] == 1:
    X_test = np.squeeze(X_test, axis=-1)

# Calcolo SSIM per ciascuna immagine
ssim_values = []
for i in range(len(X_test_reconstructed)):
    orig_img = X_test[i]
    recon_img = X_test_reconstructed[i]

    y_min, y_max, x_min, x_max = find_bounding_box(orig_img)

    orig_rect = orig_img[y_min:y_max+1, x_min:x_max+1]
    recon_rect = recon_img[y_min:y_max+1, x_min:x_max+1]

    ssim_value = ssim(orig_rect, recon_rect, data_range=1.0)
    ssim_values.append(ssim_value)
    #print(f"SSIM per immagine {i + 1}: {ssim_value}")

# Media e deviazione standard dei valori SSIM
mean_ssim = np.mean(ssim_values)
std_ssim = np.std(ssim_values)

print(f"Valore medio di SSIM con maschera: {mean_ssim}")
print(f"Deviazione standard di SSIM: {std_ssim}")


# Visualizzazione della maschera per un esempio
example_index = 557
example_orig_img = X_test[example_index]
example_recon_img = X_test_reconstructed[example_index]

y_min, y_max, x_min, x_max = find_bounding_box(example_orig_img)

fig, ax = plt.subplots(1, 2, figsize=(10, 5))

ax[0].imshow(example_orig_img, cmap='gray')
rect = patches.Rectangle((x_min, y_min), x_max-x_min, y_max-y_min, linewidth=1, edgecolor='r', facecolor='none')
ax[0].add_patch(rect)
ax[0].set_title("Originale con Maschera")
ax[0].axis('off')

ax[1].imshow(example_recon_img, cmap='gray')
ax[1].set_title("Ricostruita")
ax[1].axis('off')

plt.show()

In [ ]:
## Comparazione immagini originali e ricostruite


def plot_comparison(X_true, X_pred, num_images=10):

    indices = np.random.choice(len(X_true), num_images, replace=False)
    
    plt.figure(figsize=(15, 10))
    
    for i, idx in enumerate(indices):
        plt.subplot(num_images, 2, 2*i + 1)
        plt.imshow(X_true[idx].reshape(64, 64), cmap='gray')
        plt.title(f"Original {i+1}")
        plt.axis('off')
        
        plt.subplot(num_images, 2, 2*i + 2)
        plt.imshow(X_pred[idx].reshape(64, 64), cmap='gray')
        plt.title(f"Reconstructed {i+1}")
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()


plot_comparison(X_test, X_test_reconstructed, num_images=10)


## Modello 7 strati conv + 2 blocchi trasformer + classificatore con 3 strati densi

Valore medio di SSIM con maschera: 0.02195523895514344
Deviazione standard di SSIM: 0.030026406511328374

Mean SSIM on Test Set: 0.5675735482555746


Test Loss: 0.025829948484897614, Test Accuracy: 0.0

Confusion Matrix:
[[363  45]
 [ 69 114]]

Classification Report:
              precision    recall  f1-score   support

     Class 0       0.84      0.89      0.86       408
     Class 1       0.72      0.62      0.67       183

    accuracy                           0.81       591
   macro avg       0.78      0.76      0.77       591
weighted avg       0.80      0.81      0.80       591


Validation Loss: 1.6375044584274292
Validation Accuracy: 0.657274603843689



In [ ]:


lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-4,
    decay_steps=10000,
    decay_rate=0.95
)

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',  
    patience=50,          
    restore_best_weights=True, 
    min_delta=0.0001,
    verbose = True
)


# Definizione della loss combinata per decoder
def combined_loss_1(y_true, y_pred):
    ssim_loss = 1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred, max_val=1.0))
    mse_loss = mean_squared_error(y_true, y_pred)
    bce_loss = BinaryCrossentropy()(y_true, y_pred)
    return 0.5 * bce_loss + mse_loss + 0.5 * ssim_loss

# Transformer block
def transformer_block(x, num_heads, ff_dim, rate=0.3):
    attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=x.shape[-1])(x, x)
    attn_output = Dropout(rate)(attn_output)
    out1 = Add()([x, attn_output])  # Residual connection
    out1 = LayerNormalization(epsilon=1e-6)(out1)

    ff_output = Dense(ff_dim, activation='relu')(out1)  # Feed-forward layer
    ff_output = Dense(x.shape[-1])(ff_output)  # Riduci di nuovo alla dimensione originale (64)
    ff_output = Dropout(rate)(ff_output)

    out2 = Add()([out1, ff_output])  # Residual connection con le stesse dimensioni
    return LayerNormalization(epsilon=1e-6)(out2)


stored_values = {
    'y_true_class': None,
    'y_pred_class': None,
    'y_true_recon': None,
    'y_pred_recon': None
}

#tf.config.run_functions_eagerly(False)


def create_combined_loss(lambdadec, lambdatot):
    def combined_loss_diz(y_true, y_pred):
        if len(y_true.shape) == 2:
            stored_values['y_true_class'] = y_true
            stored_values['y_pred_class'] = y_pred
        elif len(y_true.shape) == 4:
            stored_values['y_true_recon'] = y_true
            stored_values['y_pred_recon'] = y_pred

        if stored_values['y_true_class'] is not None and stored_values['y_true_recon'] is not None:

            bce_loss = tf.keras.losses.BinaryCrossentropy()(stored_values['y_true_class'], stored_values['y_pred_class'])
            
            mse_loss = tf.keras.losses.MeanSquaredError()(stored_values['y_true_recon'], stored_values['y_pred_recon'])
            bce_loss_re = tf.keras.losses.BinaryCrossentropy()(stored_values['y_true_recon'], stored_values['y_true_recon'])
            #ssim_loss = 1 - tf.reduce_mean(tf.image.ssim(stored_values['y_true_recon'], stored_values['y_pred_recon'], max_val=1.0))
            dec_loss= lambdadec * mse_loss + (1-lambdadec) * bce_loss_re
            
            combined_loss_value = lambdatot * dec_loss + (1-lambdatot) * bce_loss
        
            stored_values['y_true_class'] = None
            stored_values['y_pred_class'] = None
            stored_values['y_true_recon'] = None
            stored_values['y_pred_recon'] = None
            
            return combined_loss_value

        return 0.0
    return combined_loss_diz


def encoder(input_shape):
    inputs = Input(shape=input_shape)
    x = Conv2D(8, (3, 3), activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(inputs)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(x)
    x = GlobalAveragePooling2D()(x)
    x = Flatten()(x)
    
    x = Dense(64, activation='relu', kernel_regularizer=l1(10e-6))(x)
    encoded = Dropout(0.4)(x)
    encoder_model = Model(inputs, encoded, name='encoder')
    return encoder_model

def decoder(output_shape):
    inputs = Input(shape=output_shape)
    x = Dense(1*1*512, activation='relu', kernel_regularizer=l1(10e-6))(inputs)
    x = Reshape((1, 1, 512))(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(x)
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(x)
    decoder_model = Model(inputs, decoded, name='decoder')
    return decoder_model


def build_classifier(encoded_shape):
    inputs = Input(shape=encoded_shape)
    x = Dense(128, activation='relu', kernel_regularizer=l1(0.001))(inputs)
    x = Dropout(0.5)(x)
    x = Dense(64, activation='relu', kernel_regularizer=l1(0.001))(inputs)
    x = Dropout(0.5)(x)
    x = Dense(32, activation='relu', kernel_regularizer=l1(0.001))(inputs)
    x = Dropout(0.5)(x)
    classification = Dense(1, activation='sigmoid')(x)
    return Model(inputs, classification, name="classifier")


input_shape = (64, 64, 1)
encoder_model = encoder(input_shape)


encoded_input = encoder_model.output
x = Reshape((1, 64))(encoded_input)  # reshape per simulare una "sequenza"
x = transformer_block(x, num_heads=8, ff_dim=512)
#x = transformer_block(x, num_heads=8, ff_dim=512)
x = transformer_block(x, num_heads=8, ff_dim=512)
x = Flatten()(x)


decoder_model = decoder((64,))
decoded = decoder_model(x)


classifier_model = build_classifier((64,))
classification = classifier_model(encoded_input)

autoencoder_classifier = Model(encoder_model.input, [decoded, classification], name="autoencoder_classifier")

optimizer = Adam(learning_rate=lr_schedule, clipvalue=1.0)


autoencoder_classifier.compile(
    optimizer=optimizer,
    #loss={"decoder": combined_loss, "classifier": "binary_crossentropy"},
    loss=create_combined_loss(0.5, 0.4),
    metrics={'classifier': 'accuracy'}
)


autoencoder_classifier.summary()

history = autoencoder_classifier.fit(
    x=X_train, 
    y={"classifier": Y_train, "decoder": X_train}, 
    epochs=350, 
    batch_size=64, 
    validation_data=(X_val, {"classifier": Y_val, "decoder": X_val}),
    #callbacks=[early_stopping]
)


# Tentativi Loss singola

## funzioni

In [ ]:

def augment_images_with_labels(images_array, labels_array, num_augmented_copies):
    if images_array.ndim == 3:
        images_array = np.expand_dims(images_array, axis=-1)
    

    datagen = ImageDataGenerator(
        horizontal_flip=True,  # Ribaltamento orizzontale
        vertical_flip=True,    # Ribaltamento verticale
        rotation_range=10,     # Rotazione casuale tra -10 e 10 gradi
        width_shift_range=0.1, # Traslazione casuale del 10% su asse x
        height_shift_range=0.1,# Traslazione casuale del 10% su asse y
        zoom_range=0.2,        # Zoom casuale tra l'80% e il 120%
        brightness_range=[0.5, 0.8]  # Modifica casuale della luminosità tra l'80% e il 120%
    )


    augmented_images = []
    augmented_labels = []

    for image, label in zip(images_array, labels_array):

        augmented_images.append(image)
        augmented_labels.append(label)

    
        image = np.expand_dims(image, 0)  
        i = 0
        for batch in datagen.flow(image, batch_size=1):
            augmented_images.append(batch[0])
            augmented_labels.append(label) 
            i += 1
            if i >= num_augmented_copies:
                break


    augmented_images_array = np.array(augmented_images)
    augmented_labels_array = np.array(augmented_labels)

  
    print("Dimensione del vettore delle immagini aumentate:", augmented_images_array.shape)
    print("Dimensione del vettore delle etichette aumentate:", augmented_labels_array.shape)

    return augmented_images_array, augmented_labels_array





from tensorflow.keras.preprocessing.image import ImageDataGenerator

def augment_images(images_array, num_augmented_copies):
    if images_array.ndim == 3:
        images_array = np.expand_dims(images_array, axis=-1)
    

    datagen = ImageDataGenerator(
        horizontal_flip=True,  # Ribaltamento orizzontale
        vertical_flip=True,    # Ribaltamento verticale
        rotation_range=10,     # Rotazione casuale tra -10 e 10 gradi
        width_shift_range=0.1, # Traslazione casuale del 10% su asse x
        height_shift_range=0.1,# Traslazione casuale del 10% su asse y
        zoom_range=0.2,        # Zoom casuale tra l'80% e il 120%
        brightness_range=[0.5, 0.8]  # Modifica casuale della luminosità tra l'80% e il 120%
    )


    augmented_images = []


    for image in images_array:
        augmented_images.append(image)
        image = np.expand_dims(image, 0)
        i = 0
        for batch in datagen.flow(image, batch_size=1):
            augmented_images.append(batch[0])
            i += 1
            if i >= num_augmented_copies:
                break


    augmented_images_array = np.array(augmented_images)

    print("Dimensione del vettore delle immagini aumentate:", augmented_images_array.shape)

    return augmented_images_array

## split e preprocessing dati

In [ ]:
# Dividi i dati in set di addestramento, validazione e test
x_train, x_val = train_test_split(X_train, test_size=0.2, random_state=32, shuffle = True)
x_train, x_test_start = train_test_split(x_train, test_size=0.2, random_state=32, shuffle = True) 


print("Dimensioni set di addestramento:", x_train.shape)
print("Dimensioni set di validazione:", x_val.shape)
print("Dimensioni set di test:", x_test_start.shape)


x_train_start = augment_images(x_train, 5)
x_val_start = augment_images(x_val, 3)

x_train_start = shuffle(x_train_start, random_state = 58)
x_val_start = shuffle(x_val_start, random_state = 99)

print(x_train.shape)
print(x_val.shape)
print(x_test_start.shape)

x_train=x_train_start.astype('float32')/255.
x_val=x_val_start.astype('float32')/255.
x_test=x_test_start.astype('float32')/255.

x_test = np.expand_dims(x_test, axis = -1)


print(x_train[0].shape)
print(x_val[0].shape)
print(x_test[0].shape)

### Preprocessing per resnet

In [ ]:
x_train = np.repeat(x_train, 3, axis=-1)
x_val = np.repeat(x_val, 3, axis=-1)
x_test = np.repeat(x_test, 3, axis=-1)

print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

x_train_resized = tf.image.resize(x_train, (224, 224), method='bicubic')
x_val_resized = tf.image.resize(x_val, (224, 224), method='bicubic')
x_test_resized = tf.image.resize(x_test, (224, 224), method='bicubic')

# Converti a numpy array
x_train = x_train_resized.numpy()
x_val = x_val_resized.numpy()
x_test = x_test_resized.numpy()

## MODELLO 1 con 6 strati conv


In [ ]:


def encoder(input_shape):
    inputs = Input(shape=input_shape)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(inputs)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = GlobalAveragePooling2D()(x)
    x = Flatten()(x)
    encoded = Dense(32, activation='relu', kernel_regularizer=l1(10e-6))(x)
    encoder_model = Model(inputs, encoded, name='encoder')
    encoder_model.summary()
    return encoder_model

def decoder(output_shape):
    inputs = Input(shape=output_shape)
    x = Dense(2*2*256, activation='relu', kernel_regularizer=l1(10e-6))(inputs)
    x = Reshape((2, 2, 256))(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
    decoder_model = Model(inputs, decoded, name='decoder')
    decoder_model.summary()
    return decoder_model


input_shape = (64, 64, 1)
encoder_model = encoder(input_shape)
decoder_model = decoder(encoder_model.output_shape[1:])

inputs = Input(shape=input_shape)
encoded = encoder_model(inputs)
decoded = decoder_model(encoded)

autoencoder_model = Model(inputs, decoded, name='autoencoder')


autoencoder_model.compile(optimizer=Adam(learning_rate=1e-3),
                          loss='binary_crossentropy')


autoencoder_model.summary()


## Modello 2 con 7 strati conv, trasformer e  custom loss

In [ ]:

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-4,
    decay_steps=35000,
    decay_rate=0.9)

def combined_loss(y_true, y_pred):
    print(y_true.shape)
    print(y_pred.shape)
    ssim_loss = 1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred, max_val=1.0))
    mse_loss = mean_squared_error(y_true, y_pred)
    bce_loss = tf.keras.losses.binary_crossentropy(y_true, y_pred)
    return (1/2) * bce_loss + (1) * mse_loss + (1/2) * ssim_loss

def transformer_block(x, num_heads, ff_dim, rate=0.3):
    attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=x.shape[-1])(x, x)
    attn_output = Dropout(rate)(attn_output)
    out1 = Add()([x, attn_output])  
    out1 = LayerNormalization(epsilon=1e-6)(out1)

    ff_output = Dense(ff_dim, activation='relu')(out1)  
    ff_output = Dense(x.shape[-1])(ff_output)
    ff_output = Dropout(rate)(ff_output)

    out2 = Add()([out1, ff_output])  
    return LayerNormalization(epsilon=1e-6)(out2)

def encoder(input_shape):
    inputs = Input(shape=input_shape)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(inputs)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = GlobalAveragePooling2D()(x)
    x = Flatten()(x)
    
    x = Dense(64, activation='relu', kernel_regularizer=l1(10e-6))(x)
    encoded = Dropout(0.4)(x)
    encoder_model = Model(inputs, encoded, name='encoder')
    return encoder_model

def decoder(output_shape):
    inputs = Input(shape=output_shape)
    x = Dense(1*1*512, activation='relu', kernel_regularizer=l1(10e-6))(inputs)
    x = Reshape((1, 1, 512))(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
    decoder_model = Model(inputs, decoded, name='decoder')
    return decoder_model


input_shape = (64, 64, 1)
encoder_model = encoder(input_shape)

encoded_input = encoder_model.output

# Transformer Block
x = Reshape((1, 64))(encoded_input)  # reshape per simulare una "sequenza"
x = transformer_block(x, num_heads=8, ff_dim=512)
x = transformer_block(x, num_heads=8, ff_dim=512)
x = transformer_block(x, num_heads=8, ff_dim=512)
x = Flatten()(x)


decoder_model = decoder((64,))
decoded = decoder_model(x)


autoencoder_model = Model(encoder_model.input, decoded, name='autoencoder_transformer')

autoencoder_model.compile(optimizer=Adam(learning_rate=lr_schedule),
                          loss=combined_loss)

autoencoder_model.summary()


## Modello con perdita focale, 5 strati conv con leaky relu. Per immagini non annerite (senza ritaglio roi)

In [ ]:

## Loss che calcola una perdita personalizzata che penalizza maggiormente gli errori sui valori centrali (0.5) 
# e favorisce la precisione per i valori estremi (0 e 1)
def focal_loss(y_true, y_pred):
    epsilon = tf.keras.backend.epsilon()
    y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)
    
    mse_loss = tf.square(y_true - y_pred)
    
    central_weight = tf.square(0.4 - y_true)
    
    weighted_loss = mse_loss * (1 + (2 * central_weight))
    
    return tf.reduce_mean(weighted_loss)


# Parametri
alpha = 0.5
fm = (5,5)
kernel_initializer = he_normal()
dim = 16

def encoder(input_shape):
    inputs = Input(shape=input_shape)
    
    x = Conv2D(dim, fm, padding='same', kernel_initializer=kernel_initializer)(inputs)
    x = LeakyReLU(alpha=alpha)(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    
    x = Conv2D(2 * dim, fm, padding='same', kernel_initializer=kernel_initializer)(x)
    x = LeakyReLU(alpha=alpha)(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    
    x = Conv2D(4 * dim, fm, padding='same', kernel_initializer=kernel_initializer)(x)
    x = LeakyReLU(alpha=alpha)(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    
    x = Conv2D(8* dim, fm, padding='same', kernel_initializer=kernel_initializer)(x)
    x = LeakyReLU(alpha=alpha)(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    
    x = Conv2D(16 * dim, fm, padding='same', kernel_initializer=kernel_initializer)(x)
    x = LeakyReLU(alpha=alpha)(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    
    x = Conv2D(32 * dim, fm, padding='same', kernel_initializer=kernel_initializer)(x)
    x = LeakyReLU(alpha=alpha)(x)
    #x = GlobalAveragePooling2D()(x)
    
    x = Flatten()(x)
    x = Dense(32, kernel_initializer=kernel_initializer, activity_regularizer=l2(0.001))(x)
    x = LeakyReLU(alpha=alpha)(x)

    encoded = Dropout(0.5)(x)
    encoder_model = Model(inputs, encoded, name='encoder')
    encoder_model.summary()
    return encoder_model

def decoder(output_shape):
    inputs = Input(shape=output_shape)
    
    x = Dense(2 * 2 * 32* dim, kernel_initializer=kernel_initializer, activity_regularizer=l2(0.001))(inputs)
    x = LeakyReLU(alpha=alpha)(x)
    
    x = Dropout(0.5)(x)
    x = Reshape((2, 2, 32 * dim))(x)
    x = UpSampling2D((2, 2))(x)
    
    x = Conv2D(32 * dim, fm, padding='same', kernel_initializer=kernel_initializer)(x)
    x = LeakyReLU(alpha=alpha)(x)
    x = UpSampling2D((2, 2))(x)
    
    x = Conv2D(16 * dim, fm, padding='same', kernel_initializer=kernel_initializer)(x)
    x = LeakyReLU(alpha=alpha)(x)
    x = UpSampling2D((2, 2))(x)
    
    x = Conv2D(8 * dim, fm, padding='same', kernel_initializer=kernel_initializer)(x)
    x = LeakyReLU(alpha=alpha)(x)
    x = UpSampling2D((2, 2))(x)
    
    x = Conv2D(4 * dim, fm, padding='same', kernel_initializer=kernel_initializer)(x)
    x = LeakyReLU(alpha=alpha)(x)
    x = UpSampling2D((2, 2))(x)
    
    x = Conv2D(2 * dim, fm, padding='same', kernel_initializer=kernel_initializer)(x)
    x = LeakyReLU(alpha=alpha)(x)
    
    x = Conv2D(dim, fm, padding='same', kernel_initializer=kernel_initializer)(x)
    x = LeakyReLU(alpha=alpha)(x)
    
    
    decoded = Conv2D(1, fm, activation='sigmoid', padding='same', kernel_initializer=kernel_initializer)(x)
    
    decoder_model = Model(inputs, decoded, name='decoder')
    decoder_model.summary()
    return decoder_model

input_shape = (64, 64, 1)
encoder_model = encoder(input_shape)
decoder_model = decoder(encoder_model.output_shape[1:])

inputs = Input(shape=input_shape)
encoded = encoder_model(inputs)
decoded = decoder_model(encoded)
autoencoder_model = Model(inputs, decoded, name='autoencoder')

autoencoder_model.compile(optimizer=AdamW(learning_rate=1e-4), loss=focal_loss)

autoencoder_model.summary()

## Modello con rete pretrainata Resnet 50 per immagini non annerite

In [ ]:
def build_encoder(input_shape, trainable_layers=10):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    # Congelo tutti i layers
    for layer in base_model.layers:
        layer.trainable = False
    
    # Sblocco gli ultimi  layers per il fine-tuning
    for layer in base_model.layers[-trainable_layers:]:
        layer.trainable = True
    
    inputs = Input(shape=input_shape)
    x = base_model(inputs, training=True)
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, kernel_initializer='he_normal', activity_regularizer=tf.keras.regularizers.l2(0.001))(x)
    x = LeakyReLU(alpha=0.1)(x)
    encoded = Dropout(0.5)(x)
    
    encoder_model = Model(inputs, encoded, name='encoder')
    encoder_model.summary()
    return encoder_model

def build_decoder(encoded_shape, original_shape):
    latent_inputs = Input(shape=encoded_shape)
    x = Dense(7 * 7 * 512, kernel_initializer='he_normal')(latent_inputs)
    x = LeakyReLU(alpha=0.1)(x)
    x = Reshape((7, 7, 512))(x)
    
    x = Conv2DTranspose(256, (3, 3), strides=(2, 2), padding='same', kernel_initializer='he_normal')(x)
    x = LeakyReLU(alpha=0.1)(x)
    
    x = Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same', kernel_initializer='he_normal')(x)
    x = LeakyReLU(alpha=0.1)(x)
    
    x = Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same', kernel_initializer='he_normal')(x)
    x = LeakyReLU(alpha=0.1)(x)
    
    x = Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same', kernel_initializer='he_normal')(x)
    x = LeakyReLU(alpha=0.1)(x)
    
    x = Conv2DTranspose(16, (3, 3), strides=(2, 2), padding='same', kernel_initializer='he_normal')(x)
    x = LeakyReLU(alpha=0.1)(x)
    
    x = Conv2DTranspose(3, (3, 3), activation='sigmoid', padding='same', kernel_initializer='he_normal')(x)
    
    decoded = Reshape(original_shape)(x)
    
    decoder_model = Model(latent_inputs, decoded, name='decoder')
    decoder_model.summary()
    return decoder_model


input_shape = (224, 224, 3)
encoded_shape = (128,)


encoder = build_encoder(input_shape)
decoder = build_decoder(encoded_shape, input_shape)


autoencoder_input = Input(shape=input_shape)
encoded_output = encoder(autoencoder_input)
decoded_output = decoder(encoded_output)

autoencoder_model = Model(autoencoder_input, decoded_output, name='autoencoder')
    

autoencoder_model.compile(optimizer=AdamW(learning_rate=1e-4), loss='mean_squared_error')

autoencoder_model.summary()


## Modello Resnet per immagini annerite

In [ ]:

def build_encoder():
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(64, 64, 3))
    encoder = Model(inputs=base_model.input, outputs=base_model.layers[-1].output)
    return encoder


def build_decoder():
    decoder_input = Input(shape=(2, 2, 2048))  # Output del ResNet bottleneck
    x = Conv2DTranspose(512, kernel_size=3, strides=2, padding='same', activation='relu')(decoder_input)
    x = Conv2DTranspose(256, kernel_size=3, strides=2, padding='same', activation='relu')(x)
    x = Conv2DTranspose(128, kernel_size=3, strides=2, padding='same', activation='relu')(x)
    x = Conv2DTranspose(64, kernel_size=3, strides=2, padding='same', activation='relu')(x)
    x = Conv2DTranspose(1, kernel_size=3, strides=2, padding='same', activation='sigmoid')(x)
    decoder = Model(inputs=decoder_input, outputs=x)
    return decoder

def build_autoencoder():
    encoder = build_encoder()
    
    # congelamento dei layer dell'encoder
    for layer in encoder.layers:
        layer.trainable = False 

    
    decoder = build_decoder()

    input_img = Input(shape=(64, 64, 1))
    encoded = encoder(input_img)
    decoded = decoder(encoded)

    autoencoder = Model(inputs=input_img, outputs=decoded)
    return autoencoder

autoencoder_model = build_autoencoder()
autoencoder_model.compile(optimizer=AdamW(learning_rate=1e-4), loss='mse')
autoencoder_model.summary()


## Training

In [ ]:


## definizione callbacks 
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    patience=30,         
    restore_best_weights=True,
    min_delta=0.0002,
    verbose = True
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.1,          
    patience=20,         
    min_delta=0.0001
)



## istogramma andamento dei valori predetti durante l'allenamento (ogni 20 epoche)
def plot_predictions_histogram(epoch, model, val_data):
    if epoch % 20 == 0:
        x_val = val_data
        x_reconstructed = model.predict(x_val)
        x_val_flat = x_val.flatten()
        x_reconstructed_flat = x_reconstructed.flatten()
        
        plt.figure(figsize=(10, 5))
        plt.hist(x_reconstructed_flat, bins=20,  alpha=0.7, color='blue', label='Ricostruzioni',range=(0.01, 1))
        plt.hist(x_val_flat, bins=20, alpha=0.5, color='red', label='Input Originali',range=(0.01, 1))
        plt.xlabel('Valore')
        plt.ylabel('Frequenza')
        plt.title(f'Istogramma delle Ricostruzioni e degli Input Originali - Epoch {epoch}')
        plt.legend()
        plt.show()
        

plot_callback = LambdaCallback(
    on_epoch_end=lambda epoch, logs: plot_predictions_histogram(epoch, autoencoder_model, x_val)
)


history=autoencoder_model.fit(x_train, x_train,
                      epochs=1000,
                      batch_size=512,
                      shuffle=True,
                      validation_data=(x_val, x_val),
                      callbacks=[early_stopping, plot_callback])

# callbacks=[early_stopping, reduce_lr, plot_callback]

In [ ]:
# salvataggio pesi modello
encoder_transformer_model = Model(encoder_model.input, x, name='encoder_transformer')

encoder_transformer_model.save_weights('encoder_transformer_decay.weights.h5')

encoder_transformer_model.save('encoder_transformer_model.h5')

## Grafici loss

In [ ]:
training_loss = history.history['loss'][3:]
validation_loss = history.history['val_loss'][3:]

plt.figure(figsize=(10, 6))
plt.plot(range(1, len(training_loss) + 1), training_loss, label='Training Loss')
plt.plot(range(1, len(validation_loss) + 1), validation_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

## visualizzazione immagini

In [ ]:
num_images = 10

reconstructed_images = autoencoder_model.predict(x_test)


print(reconstructed_images.shape)
print(x_test.shape)


plt.figure(figsize=(15, 6))
for i in range(num_images):
    # Immagine originale
    plt.subplot(2, num_images, i + 1)
    plt.imshow(x_test[i], cmap='gray')
    plt.title('Originale')
    plt.axis('off')

    # Immagine ricostruita
    plt.subplot(2, num_images, i + num_images + 1)
    plt.imshow(reconstructed_images[i], cmap='gray')
    plt.title('Ricostruita')
    plt.axis('off')

plt.tight_layout()
plt.show()




## indici di similarita immagini 1 canale

In [ ]:
## predizioni
reconstructed_images = autoencoder_model.predict(x_test)


if reconstructed_images.shape[-1] == 1:
    reconstructed_images = np.squeeze(reconstructed_images, axis=-1)
if x_test.shape[-1] == 1:
    x_test = np.squeeze(x_test, axis=-1)

ssim_values = []
for i in range(len(reconstructed_images)):
    orig_img = x_test[i]
    recon_img = reconstructed_images[i]
    ssim_value = ssim(orig_img, recon_img, data_range=1.0, multichannel=False)
    ssim_values.append(ssim_value)


mean_ssim = np.mean(ssim_values)
std_ssim = np.std(ssim_values)


print(f"Valore medio di SSIM: {mean_ssim}")
print(f"Deviazione standard di SSIM: {std_ssim}")

index = 7

plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.imshow(x_test[index], cmap='gray')
plt.title("Originale")
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(reconstructed_images[index], cmap='gray')
plt.title("Ricostruita")
plt.axis('off')

plt.show()


## ssim con maschera

In [ ]:

def find_bounding_box(image):
    # Trova i bordi del rettangolo contenente la placca
    rows = np.any(image, axis=1)
    cols = np.any(image, axis=0)
    y_min, y_max = np.where(rows)[0][[0, -1]]
    x_min, x_max = np.where(cols)[0][[0, -1]]
    return y_min, y_max, x_min, x_max

if reconstructed_images.shape[-1] == 1:
    reconstructed_images = np.squeeze(reconstructed_images, axis=-1)
if x_test.shape[-1] == 1:
    x_test = np.squeeze(x_test, axis=-1)

ssim_values = []
for i in range(len(reconstructed_images)):
    orig_img = x_test[i]
    recon_img = reconstructed_images[i]
    y_min, y_max, x_min, x_max = find_bounding_box(orig_img)

    orig_rect = orig_img[y_min:y_max+1, x_min:x_max+1]
    recon_rect = recon_img[y_min:y_max+1, x_min:x_max+1]

    ssim_value = ssim(orig_rect, recon_rect, data_range=1.0)
    ssim_values.append(ssim_value)
    #print(f"SSIM per immagine {i + 1}: {ssim_value}")


mean_ssim = np.mean(ssim_values)
std_ssim = np.std(ssim_values)


print(f"Valore medio di SSIM: {mean_ssim}")
print(f"Deviazione standard di SSIM: {std_ssim}")

# Visualizzazione della maschera per un esempio 
example_index = 557
example_orig_img = x_test[example_index]
example_recon_img = reconstructed_images[example_index]

y_min, y_max, x_min, x_max = find_bounding_box(example_orig_img)

fig, ax = plt.subplots(1, 2, figsize=(10, 5))

# Immagine originale con rettangolo
ax[0].imshow(example_orig_img, cmap='gray')
rect = patches.Rectangle((x_min, y_min), x_max-x_min, y_max-y_min, linewidth=1, edgecolor='r', facecolor='none')
ax[0].add_patch(rect)
ax[0].set_title("Originale con Maschera")
ax[0].axis('off')

# Immagine ricostruita
ax[1].imshow(example_recon_img, cmap='gray')
ax[1].set_title("Ricostruita")
ax[1].axis('off')

plt.show()

## indici di similarità 3 canali

In [ ]:
import numpy as np
from skimage.metrics import structural_similarity as ssim
from sklearn.metrics import mean_squared_error
from skimage.metrics import peak_signal_noise_ratio
from scipy.spatial.distance import cosine

if reconstructed_images.shape[-1] == 1:
    reconstructed_images = np.squeeze(reconstructed_images, axis=-1)

def calculate_ssim(original, reconstructed):
    ssim_values = []
    ssim_values_mask = []
    for i in range(len(original)):
        mask = original[i] > 0
        ssim_value_mask = ssim(original[i], reconstructed[i], data_range=1.0, channel_axis=-1, win_size=3)
        ssim_values_mask.append(ssim_value_mask)
        ssim_value = ssim(original[i], reconstructed[i], data_range=1.0, channel_axis=-1, win_size=3)
        ssim_values.append(ssim_value)
    return np.mean(ssim_values_mask), np.mean(ssim_values)

def calculate_cosine_similarity(original, reconstructed):
    cosine_values = []
    cosine_values_mask = []
    for i in range(len(original)):
        original_flat = original[i].flatten()
        reconstructed_flat = reconstructed[i].flatten()
        cosine_value = 1 - cosine(original_flat, reconstructed_flat)
        cosine_values.append(cosine_value)

    for i in range(len(original)):
        mask = original[i] > 0
        original_flat = original[i][mask].flatten()
        reconstructed_flat = reconstructed[i][mask].flatten()
        cosine_value = 1 - cosine(original_flat, reconstructed_flat)
        cosine_values_mask.append(cosine_value)
    return np.mean(cosine_values), np.mean(cosine_values_mask)



ssim_value, ssim_value_mask = calculate_ssim(x_test, reconstructed_images)
cosine_similarity_value, cosine_similarity_value_mask = calculate_cosine_similarity(x_test, reconstructed_images)


print(f'Mean SSIM: {ssim_value} and Mean SSIM with mask: {ssim_value_mask}')
print(f'Mean Cosine Similarity: {cosine_similarity_value} and Mean Cosine Similarity with mask: {cosine_similarity_value_mask}')

